# Given Parameters

**Camera Position**

Pos: $x = 500mm, y = 160mm, z =1140mm$

Rot: $ZY'X'' - roll: 100°, pitch=0°, yaw=90°$

**Intrinsic Parameters**

Res: $1280px \times 1080px$

Intr: $c_x = 636px, c_y = 548px, fx=241px, fy=238px$

**Projected Point**

$u=795px, v=467px$
$d=2.7m$


In [5]:

import numpy as np
import numpy.linalg as lin
import scipy.spatial.transform as T

# Camera parameters
i_x = -0.500  # m
i_y = -0.160  # m
i_z = -1.140  # m

i_roll  = (-99.8) #deg
i_pitch = (0)  #deg
i_yaw   = (-90.9)  #deg

res_x = 1280 #px
res_y = 1080 #px

c_x = 636.0 #px
c_y = 548.0 #px
f_x = 241.0 #px
f_y = 238.0 #px

u = 795 #px
v = 467 #px
d = 2.7 #m


def extrinsic_matrix(x,y,z,roll,pitch,yaw):
    roll = np.radians(roll)
    pitch = np.radians(pitch)
    yaw = np.radians(yaw)

    quat_roll  = T.Rotation.from_quat([np.sin(roll / 2.0), 0,                   0              , np.cos(roll / 2.0)  ]) # X
    quat_pitch = T.Rotation.from_quat([0,                  np.sin(pitch / 2.0), 0              , np.cos(pitch / 2.0) ]) # Y
    quat_yaw   = T.Rotation.from_quat([0,                  0,                   np.sin(yaw/2.0), np.cos(yaw / 2.0)   ]) # Z      
    
    # Intrinsic Rotation z'y'x'' = ZYX
    rot = quat_roll * quat_pitch * quat_yaw
    rot = rot.as_matrix()

    translation = np.zeros((4,4))
    translation[:3, :3] = rot
    
    translation[0, 3] = y
    translation[1, 3] = z
    translation[2, 3] = x
    translation[3, 3] = 1

    return translation

def intrinsic_matrix():
    intrinsic = np.array([
        [f_x, 0,   c_x, 0],
        [0,   f_y, c_y, 0],
        [0,   0,   1,   0],
        [0,   0,   0,   1]
    ])
    
    return intrinsic
    

question = np.array([[u],[v],[1],[1]])

intrinsic = intrinsic_matrix()
extrinsic = extrinsic_matrix(i_x,i_y,i_z,i_roll,i_pitch,i_yaw)

proj = intrinsic @ extrinsic
inv_proj = lin.inv(proj)

unproj_question =  inv_proj @ question
print("SOLUTION:")
print(unproj_question * d)


SOLUTION:
[[4.3230969 ]
 [2.28151352]
 [1.43823827]
 [2.7       ]]


## Resources
- https://www.mathworks.com/help/driving/ug/coordinate-systems.html
- https://danceswithcode.net/engineeringnotes/quaternions/quaternions.html
- https://docs.opencv.org/4.x/d9/d0c/group__calib3d.html (scroll down)